### Data Importing and Cleaning

#### Problem Statement

Yelp is a platform that allows users to make reservations, leave reviews, and find businesses. In addition, users can mark which reviews they find useful.  Reviews can be both very helpful and very harmful for businesses. Being able to determine what makes a useful review can help businesses create better, more  targeted listings. 



In [1]:
# imports
import pandas as pd
import sqlite3

In [2]:
# read in yelp json lines 
yelp = pd.read_json('../data/yelp.json',lines=True)

#### Review Data

In [3]:
# visualize the data
yelp.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,-P5E9BYUaK7s3PwBF5oAyg,Jha0USGDMefGFRLik_xFQg,bMratNjTG5ZFEA6hVyr-xQ,5,0,0,0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05
1,dQ3EU6cevDqHAr_ygy1O8A,CNyXcn0c0V5CFmigqqw-Xg,oY5LFo6Yxxf32ePna6mEUQ,5,1,0,0,"I absolutely love this place!\n\nGreat hours, ...",2014-12-30 17:55:51
2,Pgh9POx-bH7JFggKXqXWMQ,8fL5qUckzt_nAC1uwvbr0w,uW8L6awmCyjovD9OhWPo7g,5,1,0,1,"As far as I know, this is the best video renta...",2008-04-30 15:49:16
3,Sgs-rER85vBaOBSPVo96xw,EIi4Fy_JW_6v7DaRDet1uw,Q1HHAb4FzrzfnnrRyA8fgg,4,0,0,0,Great atmosphere and service! I don't know how...,2015-07-28 14:26:48
4,yqJv_8CoXNb-NpaEiTY4yw,ZiI40HVbRbFE-tv2K8OQkw,45siW2fI0Cuv5ZKCS23knA,5,0,0,0,Great new location on Central. Great staff and...,2014-04-19 13:06:28


In [4]:
# get shape of data
yelp.shape

(21032, 9)

In [5]:
# check for nulls
yelp.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [6]:
# check datatypes
yelp.dtypes

review_id              object
user_id                object
business_id            object
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text                   object
date           datetime64[ns]
dtype: object

In [7]:
# check review_ids are unique
yelp.nunique()

review_id      21032
user_id        19931
business_id    15494
stars              5
useful            43
funny             27
cool              38
text           21032
date           21032
dtype: int64

#### Data Preparation

In [8]:
# replace newlines with spaces & remove punctuation
yelp.text = yelp.text.str.replace('\n', ' ')
yelp.text = yelp.text.replace('[^a-zA-Z0-9 ]', '', regex=True)

In [9]:
# create target
yelp['target'] = yelp.useful.apply(lambda x: 0 if x < 1 else 1)

In [10]:
# calculate number of words in text
yelp['num_words'] = yelp.text.apply(lambda x: len(x.split(' ')))

In [11]:
# calculate number of chars in text
yelp['num_chars'] = yelp.text.apply(lambda x: len(x))

In [12]:
# check for empty texts
yelp.text.isnull().sum()

0

In [13]:
yelp.shape

(21032, 12)

#### Export Data

In [14]:
# export clean data to csv
yelp.to_csv('../data/yelp.csv',index=False)

In [15]:
# import data to db
db_connection = sqlite3.connect('../data/yelp.db.sqlite')
yelp.to_sql(
    'reviews',
    con=db_connection,
    if_exists = 'replace',
    index=False
)

In [16]:
# check numbers of rows in db
pd.read_sql('select count(*) as count from reviews;',
           con=db_connection)

,count
0,21032


### Data Dictionary

The Yelp dataset was sourced from <a href="https://www.yelp.com/dataset">here</a>.  The initial dataset contains 6,990,280 reviews in json format, from which I randomly sampled 21,032 reviews using perl.  The data that I used contains keys to other json datasets (user and business jsons), which I deemed unnecessary to the scope of this study.  For the reviews, each review json has a unique key, which I verified in the data. Business and user keys do not have to be unique.  The json file format is in json lines (each row contains a json, which are separated by new lines). 

In addition to the review key, the data also contains the number of stars a review has received.  The stars range from 1 to 5, with 5 being the best possible score.  The review json also contains "useful", "funny", and "cool" variables, which are integers that are cumulative.  In other words, they can contain any number of values.  The text string is the review itself, and the date is the date the review was posted.  In addition to the fields that came with the Yelp data, I also calculated the number of words, and the number of characters in the text variable, as well as setting the target variable (0 for no reviews, 1 for 1+ values).  

The documentation can be found <a href="https://www.yelp.com/dataset/documentation/main">here</a>. 

The shape of the dataframe is 21,032 by 12, and there are no null values in the data. The datatypes are integers for "stars", "useful", "funny", "cool", "target", "num_words", and "num_chars", while the "text" and keys are string objects.  The "date" is in datetime format.

|Feature|Type|Dataset|Description|
|---|---|---|---|
|review_id|object|Yelp|Unique review id key|
|user_id|object|Yelp|User id key of the reviewer who left the review|
|business_id|object|Yelp|Business id key of the business being reviewed|
|stars|int64|Yelp|Yelp review star rating (between 1-5 stars|
|useful|int64|Yelp|Number of "useful" votes received|
|funny|int64|Yelp|Number of "funny" votes received|
|cool|int64|Yelp|Number of "cool" votes on the review|
|text|object|Yelp|The review text|
|date|datetime64|Yelp|Date review was posted|
|target|int64|Calculated|Binary 1 = useful, 0 = not useful|
|num_words|int64|Calculated|Number of words in the review text|
|num_chars|int64|Calculated|Number of characters in the review text